

---
**IMPORTS**

---





In [ ]:
!pip install wolframalpha api

import nltk
import numpy as np
import pandas as pd
from nltk.tokenize import TweetTokenizer 
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import wolframalpha
import random
from collections import defaultdict
from nltk import ClassifierBasedTagger
from sklearn.linear_model import LogisticRegression
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction import DictVectorizer
from nltk import ChunkParserI, tree2conlltags, conlltags2tree
from nltk.metrics import accuracy
nltk.download('averaged_perceptron_tagger')

     |████████████████████████████████| 163kB 5.6MB/s 
  Created wheel for api: filename=api-0.0.7-cp36-none-any.whl size=2327 sha256=453f0d88619c43bee50bb87937afbd99e720096ea2d804b77432e54e615733b4
  Stored in directory: /root/.cache/pip/wheels/8e/b3/13/6fb6612d6dd2d4bc2dfa783e9be7554e7c2f276912d3cb850e
Successfully built api
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True



---
**CUSTOM TRAINING DATASET**

---








In [ ]:
TRAINING_SET = {
    
# General Intents

'greetings': [
  ("Hello!", {}),
  ("Hi!", {}),
  ("How are you?", {}),
  ("Hi There!", {}),
  ("Hello there!!!", {}),
  ("Ello!", {}),
  ("Hey!", {}),
  ("Hello mate, how are you?", {}),
  ("Good morning!", {}),
  ("mornin'", {}),
],

'thanks': [ 
  ("Thanks dude!", {}),
  ("Thank you!", {}),
  ("Thanks", {}),
  ("awesome, thanks for your help", {}),
  ("Thank you so much!", {}),
  ("Cheers!", {}),
  ("Cheers, thanks", {}),
  ("many thanks", {}),
],

'bye': [
  ("Bye!", {}),
  ("Bye Bye!", {}),
  ("ByeBye!", {}),
  ("Bbye!", {}), ("See you!", {}), ("See you later", {}), ("Good bye", {}), ("See you soon", {}), ("Talk to you later!", {}),
],

'rude': [ 
  ("You're an idiot", {}),
  ("You're such a fucking idiot!", {}),
  ("You're so stupid !!!", {}),
  ("You stupid fuck !!!", {}),
  ("Shut up!", {}),
  ("Shut the fuck up you fuckhead!", {}),
  ("Go to hell!", {}),
  ("shit!!", {}),
  ("You dumbass!!", {}),
  ("fuck you!!", {}),
  ("haha, you stupid !", {}),
],

'chit-chat': [ 
  ("How are you?", {}),
  ("Hey, how are you?", {}),
  ("How's life?", {}),
  ("How you've been?", {}),
  ("How's your day?", {}),
  ("How's life?", {}),
  ("How's business?", {}),
],

'ask-info': [ 
  ("Tell me more about yourself", {}),
  ("Who are you?", {}),
  ("Can you provide more info?", {}),
  ("Can I get some information?", {}),
  ("Give me more info", {}),
  ("Give me some info please", {}),
  ("I need some info", {}),
  ("I would like to know more please", {}),
  ("Tell me about you", {}),
  ("Let's talk about you", {}),
  ("Who are you?", {}),
],
 
# Song based intents 

'what-singer': [ 
  ("Who sang Sugar?", {'song': 'Sugar'}),
  ("Who sang Bohemian Rhapsody?", {'song': 'bohemian rhapsody'}),
  ("Who sang Halo?", {'song': 'Halo'}), 
  ("Who is the singer of We Are One?", {'song': 'We Are One'}), 
  ("The singer of Cold is?", {'song': 'Cold'}), 
  ("singer of Moves Like Jagger is?", {'song': 'Moves Like Jagger'}), 
  ("Who sang Killer Queen?", {'song': 'Killer Queen'}), 
  ("Who is the singer of Senorita?", {'song': 'Senorita'}),
],

'what-release-year': [ 
  ("In what year was We are the Champions released?", {'song': 'We are the Champions'}),
  ("In what year was Hello released?", {'song': 'Hello'}),
  ("When was Somebody to Love released?", {'song': 'Somebody to Love'}),
  ("When was Time is Running Out released?", {'song': 'Time is Running Out'}),
  ("When was Gangam Style produced?", {'song': 'Gangam Style'}),
  ("What's the release year for Attention?", {'song': 'Attention'}),
  ("What's the release year for Numa Numa?",  {'song': 'Numa Numa'}),
],

'get-similar-songs': [ 
  ("What are some similar songs to Memories", {'song': 'Memories'}),
  ("What are some similar song to Welcome to the Jungle?", {'song': 'Welcome to the Jungle'}),
  ("Can I get some similar song to Baby?", {'song': 'Baby'}),
  ("I want more song like Home Sweet Home ?", {'song': 'Home Sweet Home'}),
  ("I want more song like Not Afraid ?", {'song': 'Not Afraid'}),
  ("I want more song similar to Last Christmas ?", {'song': 'Last Christmas'}),
  ("I want to know more song similar to Complicated ?", {'song': 'Complicated'}),
  ("Get me more song like Numb ?", {'song': 'Numb'}),
  ("What are some song similar to In the End ?", {'song': 'In the End'}),
],

'get-latest': [ 
  ("What are some new songs?", {}),
  ("What are some new released songs?", {}),
  ("What are some new releases?", {}),
  ("Can I get some new songs?", {}),
  ("What's new?", {}),
  ("What are some new and cool songs?", {}),
  ("Tell me about some new songs?", {}),
  ("Tell me about new releases?", {}),
],

'singer-songs': [ 
  ('What songs did Ryan Gosling sing?', {'singer': 'Ryan Gosling'}),
  ('in which songs did Selena Gomez play?', {'singer': 'Selena Gomez'}),
  ('In which songs did Jason Momoa play?', {'singer': 'Jason Momoa'}),
  ('What are some songs in which Emilia Clarke acted ?', {'singer': 'Emilia Clarke'}),
  ('What are some songs which was sung by Johnny Depp ?', {'singer': 'Johnny Depp'}),
  ('Tell me some songs by Johnny Depp ?', {'singer': 'Johnny Depp'}),
  ('Tell me some songs from Tom Cruise ?', {'singer': 'Tom Cruise'}),
  ('Get me some songs sung by Leonardo DiCaprio ?', {'singer': 'Leonardo DiCaprio'}),
  ('Some songs by Matt Damon ?', {'singer': 'Matt Damon'}),
  ('Some cool songs by Keanu Reeves ?', {'singer': 'Keanu Reeves'}),
],

'song-team': [
  ("What's the team of Way Down We Go ?", {'song': 'Way Down We Go'}),
  ("What's the cast of Santa Tell Me", {'song': 'Santa Tell Me'}),
  ("Who worked in Sugar?", {'song': 'Sugar'}),
  ("Who composed City of Stars ?", {'song': 'City of Stars'}),
  ("Who produced Naked?", {'song': 'Naked'}),
  ("Who worked on Shallow?", {'song': 'Shallow'}),
],

}



---

**UTILITY CLASSES & FUNCTIONS**

---





In [ ]:
tokenizer = TweetTokenizer()
stemmer = SnowballStemmer('english')

In [ ]:
class ClassifierBasedChunkParser(ChunkParserI):
  def __init__(self, chunked_sents, feature_detector, classifier_builder, **kwargs):
    # Transform the trees in IOB annotated sentences [(word, pos, chunk), ...]
    chunked_sents = [tree2conlltags(sent) for sent in chunked_sents]
    chunked_sents = [triplets2tagged_pairs(sent) for sent in chunked_sents]
    self.feature_detector = feature_detector
    self.tagger = ClassifierBasedTaggerBatchTrained(
    train=(sent for sent in chunked_sents),
    feature_detector=self.feature_detector,
    classifier_builder=classifier_builder
    )

  def parse(self, tagged_sent):
    chunks = self.tagger.tag(tagged_sent)
    iob_triplets = tagged_pairs2triplets(chunks)
    # Transform the list of triplets to nltk.Tree format
    return conlltags2tree(iob_triplets)

  def evaluate(self, gold):
    # Convert nltk.Tree chunked sentences to (word, pos, iob) triplets
    chunked_sents = [tree2conlltags(sent) for sent in gold]
    # Convert (word, pos, iob) triplets to tagged tuples ((word, pos), iob)
    chunked_sents = [triplets2tagged_pairs(sent) for sent in chunked_sents]
    print(chunked_sents)
    dataset = self.tagger._todataset(chunked_sents)
    featuresets, tags = zip(*dataset)
    predicted_tags = self.tagger.classifier().classify_many(featuresets)
    return accuracy(tags, predicted_tags)


class ScikitClassifier(nltk.ClassifierI):
  """
  Wrapper over a scikit-learn classifier
  """
  def __init__(self, classifier=None, vectorizer=None, model=None):
    if model is None:
      if vectorizer is None:
        vectorizer = DictVectorizer(sparse=False)
      if classifier is None:
        classifier = LogisticRegression()
      self.model = Pipeline([
      ('vectorizer', vectorizer),
      ('classifier', classifier)
      ])
    else:
      self.model = model

  @property
  def vectorizer(self):
    return self.model[0][1]

  @property
  def classifier(self):
    return self.model[1][1]

  def train(self, featuresets, labels):
    self.model.fit(featuresets, labels)

  def partial_train(self, featuresets, labels, all_labels):
    self.model.partial_fit(featuresets, labels, all_labels)

  def test(self, featuresets, labels):
    self.model.score(featuresets, labels)

  def labels(self):
    return list(self.model.steps[1][1].classes_)

  def classify(self, featureset):
    return self.model.predict([featureset])[0]

  def classify_many(self, featuresets):
    return self.model.predict(featuresets)


class ClassifierBasedTaggerBatchTrained(ClassifierBasedTagger):
  def _todataset(self, tagged_sentences):
    classifier_corpus = []
    for sentence in tagged_sentences:
      history = []
      untagged_sentence, tags = zip(*sentence)
      for index in range(len(sentence)):
        featureset = self.feature_detector(untagged_sentence,
        index, history)
        classifier_corpus.append((featureset, tags[index]))
        history.append(tags[index])
    return classifier_corpus

  def _train(self, tagged_corpus, classifier_builder, verbose):
    """
    Build a new classifier, based on the given training data
    *tagged_corpus*.
    """
    if verbose:
      print('Constructing training corpus for classifier.')
    self._classifier = classifier_builder(tagged_corpus, lambda sents: self._todataset(sents))
  
  def evaluate(self, gold):
    dataset = self._todataset(gold)
    featuresets, tags = zip(*dataset)
    predicted_tags = self.classifier().classify_many(featuresets)
    return accuracy(tags, predicted_tags)


def chunk_features(tokens, index, history):
  """
  `tokens` = a POS-tagged sentence [(w1, t1), ...]
  `index` = the index of the token we want to extract features for
  `history` = the previous predicted IOB tags
  """
  # Pad the sequence with placeholders
  tokens = ([('__START2__', '__START2__'), ('__START1__', '__START1__')] +
  list(tokens) +
  [('__END1__', '__END1__'), ('__END2__', '__END2__')])
  history = ['__START2__', '__START1__'] + list(history)
  # shift the index with 2, to accommodate the padding
  index += 2
  word, pos = tokens[index]
  prevword, prevpos = tokens[index - 1]
  prevprevword, prevprevpos = tokens[index - 2]
  nextword, nextpos = tokens[index + 1]
  nextnextword, nextnextpos = tokens[index + 2]
  return {
  'word': word,
  'lemma': stemmer.stem(word),
  'pos': pos,
  'next-word': nextword,
  'next-pos': nextpos,
  'next-next-word': nextnextword,
  'nextnextpos': nextnextpos,
  'prev-word': prevword,
  'prev-pos': prevpos,
  'prev-prev-word': prevprevword,
  'prev-prev-pos': prevprevpos,
  # Historical features
  'prev-chunk': history[-1],
  'prev-prev-chunk': history[-2],
  }

def triplets2tagged_pairs(iob_sent):
  """
  Transform the triplets to tagged pairs:
  [(word1, pos1, iob1), (word2, pos2, iob2), ...] ->
  [((word1, pos1), iob1), ((word2, pos2), iob2),...]
  """
  return [((word, pos), chunk) for word, pos, chunk in iob_sent]

def tagged_pairs2triplets(iob_sent):
  """
  Transform the triplets to tagged pairs:
  [((word1, pos1), iob1), ((word2, pos2), iob2),...] ->
  [(word1, pos1, iob1), (word2, pos2, iob2), ...]
  """
  return [(word, pos, chunk) for (word, pos), chunk in iob_sent]

In [ ]:
def train_intent_model(train_set):
  pipeline = Pipeline(steps=[
  ('vectorizer', CountVectorizer(tokenizer=tokenizer.tokenize)),
  ('classifier', MultinomialNB()),
  ])
  dataset = []
  for intent, samples in train_set.items():
    for sample in samples:
      dataset.append((sample[0], intent))
  random.shuffle(dataset)
  intent_X, intent_y = zip(*dataset)
  pipeline.fit(intent_X, intent_y)
  print("Accuracy on training set:", pipeline.score(intent_X, intent_y))
  return pipeline

def sub_list(lst, slst):
  """
  Utility function for getting
  the index of a sublist inside a list
  """
  start_index = 0
  while len(lst) > len(slst):
    if lst[:len(slst)] == slst:
      return start_index
    lst, start_index = lst[1:], start_index + 1
  return None

def mark_entities(tagged_sentence, entity_words, label):
  """
  tagged_sentence: [('Word', 'Tag'), ...]
  entity_words: ['This', 'is', 'an', 'entity']
  label: the entity type
  return a nltk.Tree instance with the entities wrapped in chunks
  """
  iob_tagged = [(w, t, 'O') for w, t in tagged_sentence]
  words = nltk.untag(tagged_sentence)
  start_index = sub_list(words, entity_words)
  if start_index is not None:
    iob_tagged[start_index] = (
    iob_tagged[start_index][0],
    iob_tagged[start_index][1],
    'B-' + label
    )
    for idx in range(1, len(entity_words)):
      iob_tagged[start_index + idx] = (
      iob_tagged[start_index + idx][0],
      iob_tagged[start_index + idx][1],
      'I-' + label
      )
  return nltk.conlltags2tree(iob_tagged)

def build_extractors_datasets(train_set):
  """
  Transform the training set from the original form nltk.Tree organized by entity_type
  {entity_type: [tree1, tree2, ...]}
  """
  tokenizer = TweetTokenizer()
  datasets = defaultdict(list)
  for _, samples in train_set.items():
    for sample in samples:
      words = tokenizer.tokenize(sample[0])
      tagged = nltk.pos_tag(words)
      for entity_type, entity in sample[1].items():
        entity_words = tokenizer.tokenize(entity)
        tree = mark_entities(tagged, entity_words, entity_type)
        datasets[entity_type].append(tree)
  return datasets

def build_intent_extractor_mapping(train_set):
  mapping = {}
  for intent in train_set:
    mapping[intent] = set({})
    for sample in train_set[intent]:
      mapping[intent].update(list(sample[1].keys()))
  return mapping

def train_extractor(trees):
  def train_scikit_classifier(sentences, feature_detector):
    dataset = feature_detector(sentences)
    featuresets, labels = zip(*dataset)
    scikit_classifier = ScikitClassifier()
    scikit_classifier.train(featuresets, labels)
    return scikit_classifier
  classifier_chunker = ClassifierBasedChunkParser(
  trees,
  chunk_features,
  train_scikit_classifier,
  )
  print("Accuracy on training set:", classifier_chunker.evaluate(trees))
  return classifier_chunker



---
**CHAT BOT CLASS**

---





In [ ]:
class Chatbot(object):

  def __init__(self, min_intent_confidence=.2):
    self.min_intent_confidence = min_intent_confidence
    self.intent_model = None
    self.entity_types = []
    self.extractors = {}
    self.intent_extractor_mapping = None
    self.handlers = {}

  def train(self, training_set):
    # Train intent model
    self.intent_model = train_intent_model(training_set)
    extractor_datasets = build_extractors_datasets(training_set)
    self.intent_extractor_mapping = build_intent_extractor_mapping(training_set)
    # Train extractor for each entity type
    for entity_type, data in extractor_datasets.items():
      self.extractors[entity_type] = train_extractor(data)

  def predict_intent(self, line):
    probs = self.intent_model.predict_proba([line])[0]
    best_score_index = probs.argmax()
    best_score = probs[best_score_index]
    if best_score < self.min_intent_confidence:
      return None
    return self.intent_model.classes_[best_score_index]

  def predict_entities(self, line, intent):
    tagged = nltk.pos_tag(tokenizer.tokenize(line))
    # Get applicable extractors for the intent
    applicable_extractors = self.intent_extractor_mapping[intent]
    # Apply the extractors and get the entities
    entities = {}
    for extractor_name in applicable_extractors:
      tree = self.extractors[extractor_name].parse(tagged)
      extracted_entities = [t for t in tree if isinstance(t, nltk.Tree)]
      first_entity = None
      if extracted_entities:
        first_entity = extracted_entities[0]
      if first_entity != None:
        entities[extractor_name] = ' '.join(t[0] for t in first_entity)
    return entities

  def handle(self, intent, entities):
    assert intent in self.handlers, "Register a handler for intent: %s" % intent
    return self.handlers[intent](entities)

  def register_handler(self, intent):
    """ Decorator for registering an intent handler """
    def wrapper(handler):
      self.handlers[intent] = handler
    return wrapper

  def register_default_handler(self):
    """ Decorator for registering the default (fallback) handler """
    def wrapper(handler):
      self.handlers[None] = handler
    return wrapper

  def tell(self, line):
    intent = self.predict_intent(line)
    print("> Intent: {}".format(intent))
    # In case we haven't registered a handler for this intent
    # let's just pretend we just don't know better
    if intent not in self.handlers:
      intent = None
    if intent is not None:
      entities = self.predict_entities(line, intent)
    else:
      entities = {}
    return self.handle(intent, entities)

In [ ]:
bot = Chatbot()
bot.train(TRAINING_SET)

Accuracy on training set: 0.9519230769230769
[[(('Who', 'WP'), 'O'), (('sang', 'VBD'), 'O'), (('Sugar', 'NNP'), 'B-song'), (('?', '.'), 'O')], [(('Who', 'WP'), 'O'), (('sang', 'VBD'), 'O'), (('Bohemian', 'NNP'), 'O'), (('Rhapsody', 'NNP'), 'O'), (('?', '.'), 'O')], [(('Who', 'WP'), 'O'), (('sang', 'VBD'), 'O'), (('Halo', 'NNP'), 'B-song'), (('?', '.'), 'O')], [(('Who', 'WP'), 'O'), (('is', 'VBZ'), 'O'), (('the', 'DT'), 'O'), (('singer', 'NN'), 'O'), (('of', 'IN'), 'O'), (('We', 'PRP'), 'B-song'), (('Are', 'VBP'), 'I-song'), (('One', 'CD'), 'I-song'), (('?', '.'), 'O')], [(('The', 'DT'), 'O'), (('singer', 'NN'), 'O'), (('of', 'IN'), 'O'), (('Cold', 'NNP'), 'B-song'), (('is', 'VBZ'), 'O'), (('?', '.'), 'O')], [(('singer', 'NN'), 'O'), (('of', 'IN'), 'O'), (('Moves', 'NNS'), 'B-song'), (('Like', 'IN'), 'I-song'), (('Jagger', 'NNP'), 'I-song'), (('is', 'VBZ'), 'O'), (('?', '.'), 'O')], [(('Who', 'WP'), 'O'), (('sang', 'VBD'), 'O'), (('Killer', 'NNP'), 'B-song'), (('Queen', 'NNP'), 'I-song'



---
**GENERAL HANDLERS**

---





In [ ]:
@bot.register_default_handler()
def default_response(_):
  return "Sorry, I can't help you out with that ..."

@bot.register_handler('greetings')
def say_hello(_):
  return random.choice([
  "Hi!",
  "Hi there!",
  "Hello yourself!"
  ])

@bot.register_handler('bye')
def say_bye(_):
  return random.choice([
  "Goodbye to you!",
  "Nice meeting you, bye",
  "Have a good day!",
  "Goodbye"
  ])

@bot.register_handler('thanks')
def say_thanks(_):
  return random.choice([
  "No, thank you!",
  "It's what I do ...",
  "At your service ...",
  "Come back anytime"
  ])

@bot.register_handler('chit-chat')
def make_conversation(_):
  return random.choice([
  "I'm fine, thanks for asking ...",
  "The weather is fine, business is fine, I'm pretty well myself ...",
  "I'm bored",
  "All good, thanks",
  ])

@bot.register_handler('rude')
def be_rude(_):
  return random.choice([
  "That's pretty rude of you. Come back with some manners ...",
  "You're rude, leave me alone.",
  "That's not very nice ...",
  "You're not a nice person, are you?",
  ])

@bot.register_handler('ask-info')
def provide_info(_):
  return random.choice([
  "I know a looot of songs, try me!",
  "I can help you with songs and stuff ...",
  "I am your father!",
  "I come from the future!",
  ])



---
**INFORMATIVE HANDLERS**

---





In [ ]:
@bot.register_handler('get-latest')
def get_latest(_):
  q = 'what are 2020 songs?'
  res = client.query(q)
  print(res)
  res = next(res.results).text
  
  res = ','.join(set(list(map(lambda a: a.split('|').strip(), res.split('\n')))[1]))
  return res

@bot.register_handler('what-singer')
def what_singer(entities):
  # change handler name
  print("> Entities: {}".format(entities))
  try:
    q = 'who is the singer of {} song?'.format(entities['song'])
  except KeyError:
    return 'Can not understand you.'
  res = client.query(q)
  res = next(res.results).text
  res = ','.join(set(list(map(lambda a: a.split('|')[1].strip(), res.split('\n')))[1:2]))
  return res

@bot.register_handler('what-release-year')
def what_release_year(entities):
  try:
    q = 'who is the singer of {} song?'.format(entities['song'])
  except KeyError:
    return 'Can not understand you.'

  res = client.query(q)
  res = next(res.results).text
  res = list(map(lambda a: a.split('|')[2].strip(), res.split('\n')))[1].split(',')[-1]

  return res

@bot.register_handler('get-similar-songs')
def get_similar_songs(entities):
  # rename handler name
  #
  # get name of an actor
  try:
    q1 = 'who is the singer of {} song?'.format(entities['song'])
  except KeyError:
    return 'Can not understand you.'
  # get from query
  res = client.query(q1)
  res = next(res.results).text
  actor = list(map(lambda a: a.split('|')[1].strip(), res.split('\n')))[1]
  # pass request to wolfram and get list of songs
  q2 = 'what are {} songs?'.format(actor)
  res = client.query(q2)
  res = next(res.results).text
  res = ','.join(list(filter(lambda a: a != entities['song'], map(lambda a: a.split('|')[0].strip(), res.split('\n')[1:])))[:-1])
  return res

@bot.register_handler('singer-songs')
def get_actor_songs(entities):
  # change handler name
  try:
    q = 'what are {} songs?'.format(entities['actor'])
  except KeyError:
    return 'Can not understand you.'
  # -- code --
  res = client.query(q)
  res = next(res.results).text
  res = ','.join(list(filter(lambda a: a != entities['song'], map(lambda a: a.split('|')[0].strip(), res.split('\n')))))
  return res

@bot.register_handler('song-team')
def song_cast(entities):
  # ( !! ) similar to prev one 
  # change handler name to 'get_actors'
  try:
    q = 'who is the singer of {} song?'.format(entities['song'])
  except KeyError:
    return 'Can not understand you.'
  # -- code -- 
  res = client.query(q)
  res = next(res.results).text
  res = ','.join(set(list(map(lambda a: a.split('|')[1].strip(), res.split('\n')))[1:]))
  return res



---

**WOLFRAM API SETUP**

---





In [ ]:
client = wolframalpha.Client('TLRG9X-TTY79PQ7K8')



---
**DEMO**

---







In [ ]:
while True:
   line = input("You: ")
   print("SongBot:", bot.tell(line).strip(), "\n")

You: Can I get some similar song to Halo?
> Intent: get-similar-songs
SongBot: Say My Name,Formation,Telephone,Irreplaceable,Bootylicious,Drunk in Love,XO,Listen,Survivor 

You: bye
> Intent: bye
SongBot: Goodbye to you! 



KeyboardInterrupt: ignored